# Quantization

In [ ]:
!git clone -b v0.4.6 https://github.com/intel/auto-round.git

Cloning into 'auto-round'...
remote: Enumerating objects: 5805, done.
remote: Counting objects: 100% (1987/1987), done.
remote: Compressing objects: 100% (801/801), done.
remote: Total 5805 (delta 1730), reused 1198 (delta 1186), pack-reused 3818 (from 2)
Receiving objects: 100% (5805/5805), 10.71 MiB | 14.96 MiB/s, done.
Resolving deltas: 100% (3886/3886), done.
Note: switching to '1320752144161a098cf7d221ccb777d2e7c533e3'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [ ]:
with open("/content/auto-round/auto_round/calib_dataset.py", "r") as f:
    data = f.read()

data = data.replace("dataset = dataset.filter(filter_func)", "dataset = dataset.filter(filter_func, batch_size=64)")

with open("/content/auto-round/auto_round/calib_dataset.py", "w") as f:
    f.write(data)

In [ ]:
!cd ./auto-round && pip install .[gpu]

Processing /content/auto-round
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.0 MB/s eta 0:00

In [ ]:
!auto-round-best \
  --model bigscience/bloomz-1b1 \
  --bits 4 \
  --group_size 128 \
  --nsamples 128 \
  --scale_dtype bf16 \
  --model_dtype bf16 \
  --output_dir ./models \
  --format "auto_gptq"

# Evaluation

In [1]:
!pip install lm_eval "autoawq>0.1.8" gptqmodel==2.1.0 optimum

  Using cached lm_eval-0.4.8-py3-none-any.whl.metadata (50 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.5/280.5 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached datasets-3.4.1-py3-none-any.whl.metadata (19 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached jsonlines-4.0.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
  Using cached pytablewriter-1.2.1-py3-none-any.whl.metadata (38 kB)
  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Using cached sacrebleu-2.5.1-py3-none-any.whl.metadata (51 kB)
  Usi

In [ ]:
from lm_eval import simple_evaluate
import os

os.environ["HF_ALLOW_CODE_EVAL"] = "1"

res = simple_evaluate(
    model="hf",
    model_args='pretrained=bigscience/bloomz-1b1,dtype=float16',
    tasks="humaneval,xnli_en,xwinograd_en,xstorycloze_en".split(","),
    device="cuda:0",
    batch_size=8,
    confirm_run_unsafe_code=True
)

from lm_eval.utils import make_table  # pylint: disable=E0401
print(make_table(res))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.13G [00:00<?, ?B/s]

Running generate_until requests: 100%|██████████| 164/164 [11:37<00:00,  4.25s/it]


|    Tasks     |Version|  Filter   |n-shot|Metric|   |Value |   |Stderr|
|--------------|------:|-----------|-----:|------|---|-----:|---|-----:|
|humaneval     |      1|create_test|     0|pass@1|   |0.0000|±  |0.0000|
|xnli_en       |      1|none       |     0|acc   |↑  |0.4811|±  |0.0100|
|xstorycloze_en|      1|none       |     0|acc   |↑  |0.6446|±  |0.0123|
|xwinograd_en  |      1|none       |     0|acc   |↑  |0.7286|±  |0.0092|



In [1]:
from lm_eval import simple_evaluate
import os

os.environ["HF_ALLOW_CODE_EVAL"] = "1"

res = simple_evaluate(
    model="hf",
    model_args='pretrained=itdainb/bloomz-1b1-w4g128-auto-gptq,dtype=float16',
    tasks="humaneval,xnli_en,xwinograd_en,xstorycloze_en".split(","),
    device="cuda:0",
    batch_size=8,
    confirm_run_unsafe_code=True
)

from lm_eval.utils import make_table  # pylint: disable=E0401
print(make_table(res))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

...

0000.parquet:   0%|          | 0.00/357k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/360 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/1511 [00:00<?, ? examples/s]

Running generate_until requests: 100%|██████████| 164/164 [13:18<00:00,  4.87s/it]


|    Tasks     |Version|  Filter   |n-shot|Metric|   |Value |   |Stderr|
|--------------|------:|-----------|-----:|------|---|-----:|---|-----:|
|humaneval     |      1|create_test|     0|pass@1|   |0.0000|±  |0.0000|
|xnli_en       |      1|none       |     0|acc   |↑  |0.4952|±  |0.0100|
|xstorycloze_en|      1|none       |     0|acc   |↑  |0.6406|±  |0.0123|
|xwinograd_en  |      1|none       |     0|acc   |↑  |0.7256|±  |0.0093|



In [1]:
!pip install gputil gptqmodel==2.1.0 optimum

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.5/280.5 kB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.6 MB/s eta 0:

In [1]:
import torch
import time
import GPUtil
import numpy as np
import traceback
import threading
from transformers import TextIteratorStreamer
from typing import Optional, Dict, Any, List, Union

class GPUMonitor:
    """
    GPU monitoring class using GPUtil.
    Tracks peak GPU memory and utilization.
    """
    def __init__(self, monitoring_interval: float = 0.1):
        self.monitoring_interval = monitoring_interval
        self._gpu_memory_usage = []
        self._gpu_utilization = []
        self._is_monitoring = False
        self._monitoring_thread = None

    def start(self):
        """Start GPU monitoring"""
        self._is_monitoring = True
        self._gpu_memory_usage = []
        self._gpu_utilization = []

        def monitor_gpu():
            while self._is_monitoring:
                try:
                    # Get GPU information
                    gpus = GPUtil.getGPUs()
                    if gpus:
                        gpu = gpus[0]  # Assuming first GPU
                        self._gpu_memory_usage.append(gpu.memoryUsed)
                        self._gpu_utilization.append(gpu.load * 100)

                    # Wait for next interval
                    time.sleep(self.monitoring_interval)
                except Exception as e:
                    print(f"GPU monitoring error: {e}")
                    break

        self._monitoring_thread = threading.Thread(target=monitor_gpu)
        self._monitoring_thread.start()

    def stop(self):
        """Stop GPU monitoring"""
        self._is_monitoring = False
        if self._monitoring_thread:
            self._monitoring_thread.join()

    def get_peak_usage(self) -> float:
        """Get peak GPU memory usage in MB"""
        return max(self._gpu_memory_usage) if self._gpu_memory_usage else 0

    def get_peak_utilization(self) -> float:
        """Get peak GPU utilization percentage"""
        return max(self._gpu_utilization) if self._gpu_utilization else 0

    def get_p90_usage(self) -> float:
        """Get P90 GPU memory usage in MB"""
        return np.percentile(self._gpu_memory_usage, 90) if self._gpu_memory_usage else 0

    def get_p90_utilization(self) -> float:
        """Get P90 GPU utilization percentage"""
        return np.percentile(self._gpu_utilization, 90) if self._gpu_utilization else 0

def benchmark_single_prompt(
        model,
        tokenizer,
        input_prompt_text: str,
        temperature: float = 1.0,
        top_p: float = 0.95,
        max_new_tokens: int = 100,
        device: Optional[str] = None) -> Dict[str, Any]:
    """
    Benchmark a language model's performance for a single prompt.
    """
    # Determine device
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)

    # GPU monitoring setup
    gpu_monitor = GPUMonitor()
    gpu_monitor.start()

    # Tokenize input
    start_input_process = time.time()
    inputs = tokenizer(input_prompt_text, return_tensors="pt").to(device)
    input_process_time = time.time() - start_input_process

    # Streaming generation setup
    streamer = TextIteratorStreamer(tokenizer, skip_special_tokens=False)
    generation_start_time = time.time()
    first_token_time = None
    generated_decoded_tokens = []

    # Streaming generation loop
    try:
        generation_kwargs = {
            'input_ids': inputs.input_ids,
            'attention_mask': inputs.attention_mask,
            'max_new_tokens': max_new_tokens,
            'temperature': temperature,
            'top_p': top_p,
            'do_sample': temperature > 0
        }

        def generate():
            model.generate(**generation_kwargs, streamer=streamer)

        generation_thread = threading.Thread(target=generate)
        generation_thread.start()

        for token in streamer:
            if first_token_time is None:
                first_token_time = time.time() - generation_start_time
                first_token_start_time = time.time()
            generated_decoded_tokens.append(token)

    except Exception as e:
        print(f"Generation error: {e}")
        print(f"Error trace:\n{traceback.format_exc()}")
        return {}

    # Stop GPU monitoring
    gpu_monitor.stop()

    # Generation metrics
    generation_time = time.time() - first_token_start_time
    total_generation_time = time.time() - generation_start_time

    # Calculate metrics
    input_tokens = inputs.input_ids.shape[1]
    output_tokens = len(generated_decoded_tokens)
    total_tokens = input_tokens + output_tokens

    # Get GPU metrics
    peak_gpu_usage = gpu_monitor.get_peak_usage()
    peak_gpu_utilization = gpu_monitor.get_peak_utilization()
    p90_gpu_usage = gpu_monitor.get_p90_usage()
    p90_gpu_utilization = gpu_monitor.get_p90_utilization()

    benchmark_results = {
        'total_time_seconds': total_generation_time,
        'time_to_first_token_seconds': first_token_time,
        'input_tokens': input_tokens,
        'output_tokens': output_tokens,
        'total_tokens': total_tokens,
        'tokens_per_second': total_tokens / total_generation_time,
        'output_decode_tokens_per_second': (output_tokens - input_tokens) / generation_time,
        'input_process_time_seconds': input_process_time,
        'peak_gpu_memory_mb': peak_gpu_usage,
        'p90_gpu_memory_mb': p90_gpu_usage,
        'peak_gpu_utilization': peak_gpu_utilization,
        'p90_gpu_utilization': p90_gpu_utilization
    }

    return benchmark_results

def benchmark_language_model(
        model,
        tokenizer,
        prompts: List[str],
        temperature: float = 1.0,
        top_p: float = 0.95,
        max_new_tokens: int = 100,
        device: Optional[str] = None) -> Dict[str, Union[float, List[Dict[str, Any]]]]:
    """
    Benchmark a language model's performance across multiple prompts.
    """
    prompt_results = []
    for prompt in prompts:
        result = benchmark_single_prompt(
            model,
            tokenizer,
            prompt,
            temperature,
            top_p,
            max_new_tokens,
            device
        )
        if result:
            prompt_results.append(result)

    if not prompt_results:
        return {}

    # Extract metric lists for aggregation
    tps_list = [result['tokens_per_second'] for result in prompt_results]
    decode_tps_list = [result['output_decode_tokens_per_second'] for result in prompt_results]
    ttft_list = [result['time_to_first_token_seconds'] for result in prompt_results]
    gpu_usage_list = [result['peak_gpu_memory_mb'] for result in prompt_results]
    gpu_util_list = [result['peak_gpu_utilization'] for result in prompt_results]

    # Aggregate metrics
    aggregate_results = {
        # Total Tokens Per Second (TPS) metrics
        'p50_total_tps': round(np.percentile(tps_list, 50), 3),
        'p90_total_tps': round(np.percentile(tps_list, 90), 3),

        # Output Decode Tokens Per Second metrics
        'p50_decode_tps': round(np.percentile(decode_tps_list, 50), 3),
        'p90_decode_tps': round(np.percentile(decode_tps_list, 90), 3),

        # Time to First Token (TTFT) metrics
        'p50_ttft_seconds': round(np.percentile(ttft_list, 50), 3),
        'p90_ttft_seconds': round(np.percentile(ttft_list, 90), 3),

        # GPU Memory Usage metrics
        'max_gpu_memory_mb': round(max(gpu_usage_list), 3),
        'p90_gpu_memory_mb': round(np.percentile(gpu_usage_list, 90), 3),

        # GPU Utilization metrics
        'max_gpu_utilization': round(max(gpu_util_list), 3),
        'p90_gpu_utilization': round(np.percentile(gpu_util_list, 90), 3)
    }

    return aggregate_results

In [2]:
base_model = "bigscience/bloomz-1b1"
quantized_model = "itdainb/bloomz-1b1-w4g128-auto-gptq"

prompt_list = [
    "Hello! What's the breaking news of the day?",
    "Write a python code to add two integers",
    "Write a 200-word essay on Global Economy.",
    "Plan a 5-day healthy meal plan for a 25-year-old woman.",
    "What are the top 5 tourist destinations in Europe?",
    "Generate a short bedtime story for kids.",
    "Explain quantum computing in simple terms.",
    "Provide a step-by-step guide to meditation for beginners.",
    "List 10 fun indoor activities for children.",
    "Summarize the plot of 'To Kill a Mockingbird'.",
    "Give me 5 tips for improving public speaking skills.",
    "Write a motivational quote about success.",
    "What are the benefits of intermittent fasting?",
    "Create a beginner's workout routine for weight loss.",
    "Write a short biography of Albert Einstein.",
    "Explain the Pythagorean Theorem with an example.",
    "Give me a list of top 10 programming languages in 2025.",
    "How can I improve my sleep quality naturally?",
    "Write a simple Python program to check if a number is prime.",
    "List 10 habits of highly productive people.",
    "What are some effective strategies for learning a new language?",
    "Explain blockchain technology in layman’s terms.",
    "Describe the impact of social media on mental health.",
    "Write a short poem about nature."
]


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [4]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model, torch_dtype=torch.float16, device_map={"": "cuda"})

base_bench_result = benchmark_language_model(model, tokenizer, prompt_list)
base_bench_result

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'p50_total_tps': np.float64(52.813),
 'p90_total_tps': np.float64(120.742),
 'p50_decode_tps': np.float64(22.992),
 'p90_decode_tps': np.float64(33.487),
 'p50_ttft_seconds': np.float64(0.002),
 'p90_ttft_seconds': np.float64(0.003),
 'max_gpu_memory_mb': 2232.0,
 'p90_gpu_memory_mb': np.float64(2232.0),
 'max_gpu_utilization': 51.0,
 'p90_gpu_utilization': np.float64(48.0)}

In [4]:
tokenizer = AutoTokenizer.from_pretrained(quantized_model)
model = AutoModelForCausalLM.from_pretrained(quantized_model, torch_dtype=torch.float16, device_map={"": "cuda"})

quant_bench_result = benchmark_language_model(model, tokenizer, prompt_list)
quant_bench_result

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.      
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.                              
INFO   Kernel: Auto-selection: adding candidate `ExllamaQuantLinear`                               


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


INFO  Format: Converting `checkpoint_format` from `gptq` to internal `gptq_v2`.                    
INFO  Format: Converting GPTQ v1 to v2                                                             
INFO  Format: Conversion complete: 0.034981727600097656s                                           


{'p50_total_tps': np.float64(79.552),
 'p90_total_tps': np.float64(119.646),
 'p50_decode_tps': np.float64(-31.095),
 'p90_decode_tps': np.float64(2.375),
 'p50_ttft_seconds': np.float64(0.003),
 'p90_ttft_seconds': np.float64(0.011),
 'max_gpu_memory_mb': 1258.0,
 'p90_gpu_memory_mb': np.float64(1258.0),
 'max_gpu_utilization': 49.0,
 'p90_gpu_utilization': np.float64(42.7)}

# Report

## Model Performance Comparison

| Models      | Task             | Metric | ↑  | Value  | ±   | Stderr  | Runtime (m) | Size (GB) |
|------------|----------------|--------|----|--------|----|--------|-------------|---------|
| **FP16**    | xnli_en        | acc    | ↑  | 0.4811 | ±  | 0.0100 | -           | -       |
|            | xstorycloze_en | acc    | ↑  | 0.6446 | ±  | 0.0123 | 13:11       | 2.13    |
|            | xwinograd_en   | acc    | ↑  | 0.7286 | ±  | 0.0092 | -           | -       |
|------------|----------------|--------|----|--------|----|--------|-------------|---------|
| **GPTQ 4-bit** | xnli_en    | acc    | ↑  | 0.4952 | ±  | 0.0100 | -           | -       |
|            | xstorycloze_en | acc    | ↑  | 0.6406 | ±  | 0.0123 | 15:02       | 1.13    |
|            | xwinograd_en   | acc    | ↑  | 0.7256 | ±  | 0.0093 | -           | -       |


## Performance Metrics Comparison

| Metric                  | FP16        | GPTQ 4-bit  |
|-------------------------|------------|------------|
| **p50_total_tps**       | 52.813     | 79.552     |
| **p90_total_tps**       | 120.742    | 119.646    |
| **p50_decode_tps**      | 22.992     | -31.095    |
| **p90_decode_tps**      | 33.487     | 2.375      |
| **p50_ttft_seconds**    | 0.002      | 0.003      |
| **p90_ttft_seconds**    | 0.003      | 0.011      |
| **max_gpu_memory_mb**   | 2232.0     | 1258.0     |
| **p90_gpu_memory_mb**   | 2232.0     | 1258.0     |
| **max_gpu_utilization** | 51.0       | 49.0       |
| **p90_gpu_utilization** | 48.0       | 42.7       |
